### Verifying the forecasting of a time series

In [22]:
## Fault Injection
import numpy as np
import pandas as pd
from numpy import array
from pandas import read_csv
from pandas import DataFrame
import matplotlib.pyplot as plt
from random import randint
# from random import random
import random
import math



# def randomised(data):
#     n = len(data)
#     fault_units = [-30.5, -20.5, 0, 15.5, 25.5]
#     for col in data.columns:
        
#         for i in range(len(data)):
#             p = random.random()
#             if p > 0.5:
#                 data[col].iloc[i] += random.choice(fault_units)

#     return data


def value_flip(data, loc, col):
    for dp in data.index[loc]:
        if(data.loc[dp, col] == 1):
            data.loc[dp, col] = 0
        else:
            data.loc[dp, col] = 1            
    return data


def stuck_at(data, loc, col, val):
    # This is either stuck_at zero or at 1
    for dp in data.index[loc]:
        data.loc[dp, col] = val
    return data

def insert_outlier(data, loc, col):
#     // This inserts at the beginning of the data
    thresh = random.uniform(0.5, 1.5)
    q1 = np.quantile(data, 0.25)
    q3 = np.quantile(data, 0.75)
    iqr = q3 - q1
    high_outlier = q3 + 1.5*iqr
    low_outlier = q1 - 1.5*iqr - thresh
    for dp in data.index[loc]:
        data.loc[dp, col] =high_outlier + random.uniform(0.5, 1.5)
#         data.loc[dp, col] =low_outlier - random.uniform(0.5, 1.5)
    return data


def stuck2_at(data, val, col, loc="beg"):
    # This is either stuck_at zero or at 1
    if(loc == "beg"):
        n = math.floor(len(data[col])*0.3)
        for i in range(n):
            data[col][i] = val
    return data

# mydata = read_csv('simulated_waterTank.csv', header=0, index_col=0, skiprows=4)
mydata = read_csv('waterTankNew.csv', header=0, index_col=0)

faulty_data = stuck_at(mydata, range(500,700), "Tank1InFlow", 1)
faulty_data.to_csv('stuck_at_faulty_data.csv')

faulty_data = stuck_at(mydata, range(500,700), "Tank1InFlow", 0)
faulty_data.to_csv('stuck_at_zero_faulty_data.csv')


faulty_data_vf = value_flip(mydata, range(500,700), "Tank1InFlow")
faulty_data_vf.to_csv('value_flip_faulty_data.csv')

faulty_data_outlier = insert_outlier(mydata, range(500,700), "Tank1InFlow")
faulty_data_outlier.to_csv('outlier_faulty_data.csv')
# faulty_data.head()
# mydata

print("Files saved successfully")

Files saved successfully


### Prediction with faulty data (stuck_at)

In [16]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
s_model = load_model('Enc-Dec-lstm.h5')

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


data = read_csv('stuck_at_faulty_data.csv', header=0, index_col=0)

# horizontally stack columns
raw_data = data.values
print(f'The shape of the raw_data before scaling is {raw_data.shape}')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
f_scaled_data = scaler.fit_transform(raw_data)

# n_train = 10000
# dataset = f_scaled_data[0:n_train,:]


new_data = f_scaled_data[0:2000,:]

# choose a number of time steps
n_steps_in, n_steps_out = 50, 100

# convert into input/output
test_X, test_y = split_sequences(new_data, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = test_X.shape[2]

score = s_model.evaluate(test_X, test_y, verbose=0)
test_yhat = s_model.predict(test_X, verbose=0)
print("%s:  %.2f%%" % (s_model.metrics_names[0], score*100))

print("Prediction is successful!")

The shape of the raw_data before scaling is (20001, 5)
loss:  7.28%
Prediction is successful!


### Prediction with faulty data (value_flip)

In [17]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
s_model = load_model('Enc-Dec-lstm.h5')

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


data = read_csv('value_flip_faulty_data.csv', header=0, index_col=0)

# horizontally stack columns
raw_data = data.values
print(f'The shape of the raw_data before scaling is {raw_data.shape}')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
f_scaled_data = scaler.fit_transform(raw_data)

# n_train = 10000
# dataset = f_scaled_data[0:n_train,:]


new_data = f_scaled_data[0:2000,:]

# choose a number of time steps
n_steps_in, n_steps_out = 50, 100

# convert into input/output
test_X, test_y = split_sequences(new_data, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = test_X.shape[2]

score = s_model.evaluate(test_X, test_y, verbose=0)
test_yhat = s_model.predict(test_X, verbose=0)
print("%s:  %.2f%%" % (s_model.metrics_names[0], score*100))

print("Prediction is successful!")

The shape of the raw_data before scaling is (20001, 5)
loss:  7.60%
Prediction is successful!


### Prediction with faulty data (Outlier)

In [21]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
s_model = load_model('Enc-Dec-lstm.h5')

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


data = read_csv('outlier_faulty_data.csv', header=0, index_col=0)

# horizontally stack columns
raw_data = data.values
print(f'The shape of the raw_data before scaling is {raw_data.shape}')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
f_scaled_data = scaler.fit_transform(raw_data)

# n_train = 10000
# dataset = f_scaled_data[0:n_train,:]


new_data = f_scaled_data[0:2000,:]

# choose a number of time steps
n_steps_in, n_steps_out = 50, 100

# convert into input/output
test_X, test_y = split_sequences(new_data, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = test_X.shape[2]

score = s_model.evaluate(test_X, test_y, verbose=0)
test_yhat = s_model.predict(test_X, verbose=0)
print("%s:  %.2f%%" % (s_model.metrics_names[0], score*100))

print("Prediction is successful!")

The shape of the raw_data before scaling is (20001, 5)
loss:  6.88%
Prediction is successful!


### Prediction with faulty data (Outlier)

In [23]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
s_model = load_model('Enc-Dec-lstm.h5')

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


data = read_csv('stuck_at_zero_faulty_data.csv', header=0, index_col=0)

# horizontally stack columns
raw_data = data.values
print(f'The shape of the raw_data before scaling is {raw_data.shape}')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
f_scaled_data = scaler.fit_transform(raw_data)

# n_train = 10000
# dataset = f_scaled_data[0:n_train,:]


new_data = f_scaled_data[0:2000,:]

# choose a number of time steps
n_steps_in, n_steps_out = 50, 100

# convert into input/output
test_X, test_y = split_sequences(new_data, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = test_X.shape[2]

score = s_model.evaluate(test_X, test_y, verbose=0)
test_yhat = s_model.predict(test_X, verbose=0)
print("%s:  %.2f%%" % (s_model.metrics_names[0], score*100))

print("Prediction is successful!")

The shape of the raw_data before scaling is (20001, 5)
loss:  7.60%
Prediction is successful!


### Second Model